# Combine data from various sources
Combine the data from two ETDA sources and MITRE ATT&CK into one set.

In [1]:
import pandas as pd
import requests

#### Load ETDA actors JSON.
This data contains information on alternative names, targeted industries, countries of origin and operation dates. Since the information of targeted industries is less structured than the MISP data, only the actor names, IDs and operation dates are taken.

In [2]:
etda_actors = requests.get('https://apt.etda.or.th/cgi-bin/getcard.cgi?g=all&o=j')
data = etda_actors.json()

actor_list = []

for actor in data['values']:
    actor_dict = {}
    actor_dict['etda_uuid'] = actor['uuid']
    date_list = []
    year_list = []
    if 'operations' in actor.keys():
        for operation in actor['operations']:
            date_list.append(operation['date'])
            year_list.append(operation['date'][:4])
    actor_dict['operation_dates'] = date_list
    actor_dict['operation_year'] = year_list
    actor_list.append(actor_dict)

etda_df = pd.DataFrame(actor_list)
etda_df.head()

,etda_uuid,operation_dates,operation_year
0,4777aa1a-1bc4-4f81-8c1d-9c50c739d314,[],[]
1,fb8e2216-5e84-49ad-b187-58b457cfa95f,[],[]
2,93000251-ee2f-4865-9972-3ffe37bc62dd,[],[]
3,232c92dc-f380-4291-beca-a2e59b502449,[],[]
4,e7fb49f3-44b5-427d-93fc-570fcd7b4573,[],[]


#### Load MITRE and ETDA merge
This merge is based on tropChaud's [Catergorized Adversary TTPs](https://github.com/tropChaud/Categorized-Adversary-TTPs). Where the original method only includes actors that have known TTPs at MITRE, the change allows for the full ETDA dataset to be present and have the ATT&CK information where possible.

In [3]:
# Run file to create dataset
! python ttpCategory.py

18989a3d-e984-40a9-8fd6-ed6f6b318272 not present in ETDA group cards
438f216f-2ba2-40d1-ae92-0a3919689bae not present in ETDA group cards
960355e3-27d8-4e62-a4cc-a807f031865f not present in ETDA group cards
842c8719-1e95-473f-9d0b-6e46cfe657bf not present in ETDA group cards
8823ab67-aed0-4dd0-8425-b72db5f13952 not present in ETDA group cards


In [4]:
etda_attck_df = pd.read_json('ETDA_ATTCK_merge.json',encoding='utf-8')
etda_attck_df.head()

,mitre_attack_id,mitre_attack_name,mitre_attack_aliases,mitre_attack_created,mitre_attack_last_modified,mitre_url,etda_id,etda_name,etda_aliases,etda_first_seen,etda_url,etda_operation_dates,etda_operation_year,country,motivation,victim_industries,victim_countries,mitre_attack_ttps
0,intrusion-set--c93fccb1-e8e8-42cf-ae33-2ad1d18...,Lazarus Group,"[Lazarus Group, Labyrinth Chollima, HIDDEN COB...",2017-05-31,2022-08-23,https://attack.mitre.org/groups/G0032,41dcfaff-d5f0-484d-8649-ef8c61588eec,"Lazarus Group, Hidden Cobra, Labyrinth Chollima","[Lazarus Group, Labyrinth Chollima, Group 77, ...",2007,https://apt.etda.or.th/cgi-bin/showcard.cgi?u=...,"[2007, 2009-07, 2011-03, 2013-03, 2013-05, 201...","[2007, 2009, 2011, 2013, 2013, 2013, 2014, 201...",KP,"[Information theft and espionage, Sabotage and...","[Aerospace, Defense, Energy, Engineering, Fina...","[Australia, Bangladesh, Belgium, Brazil, Canad...","[T1573.001, T1070.003, T1588.002, T1560.002, T..."
1,intrusion-set--269e8108-68c6-4f99-b911-14b2e76...,MuddyWater,"[MuddyWater, Earth Vetala, MERCURY, Static Kit...",2018-04-18,2022-10-17,https://attack.mitre.org/groups/G0069,0d5af1f9-fa2e-4ce9-a4ce-0c6fade938e9,"MuddyWater, Seedworm, TEMP.Zagros, Static Kitten","[MuddyWater, Seedworm, TEMP.Zagros, Static Kit...",2017,https://apt.etda.or.th/cgi-bin/showcard.cgi?u=...,"[2017-02, 2018-01, 2018-03, 2018-05, 2018-05, ...","[2017, 2018, 2018, 2018, 2018, 2018, 2018, 201...",IR,[Information theft and espionage],"[Defense, Education, Energy, Financial, Food a...","[Afghanistan, Armenia, Austria, Azerbaijan, Ba...","[T1059.007, T1057, T1087.002, T1555, T1105, T1..."
2,intrusion-set--222fbd21-fc4f-4b7e-9f85-0e6e3a7...,menuPass,"[menuPass, Cicada, POTASSIUM, Stone Panda, APT...",2017-05-31,2022-07-20,https://attack.mitre.org/groups/G0045,2aa9ca75-fa1b-422e-9677-02983934f983,"Stone Panda, APT 10, menuPass","[Stone Panda, APT 10, menuPass Team, menuPass,...",2006,https://apt.etda.or.th/cgi-bin/showcard.cgi?u=...,"[2016-09, 2016, 2016/2017, 2017-02, 2017, 2017...","[2016, 2016, 2016, 2017, 2017, 2017, 2017, 201...",CN,[Information theft and espionage],"[Aerospace, Defense, Energy, Financial, Govern...","[Australia, Belgium, Brazil, Canada, China, Fi...","[T1574.001, T1090.002, T1039, T1210, T1199, T1..."
3,intrusion-set--23b6a0f5-fa95-46f9-a6f3-4549c5e...,Night Dragon,[Night Dragon],2017-05-31,2022-09-22,https://attack.mitre.org/groups/G0014,4feaae3b-b420-4bd0-ad22-1eccf413d53b,Night Dragon,[Night Dragon],2009,https://apt.etda.or.th/cgi-bin/showcard.cgi?u=...,[],[],CN,[Information theft and espionage],"[Energy, Oil and gas, Petrochemical]","[Greece, Kazakhstan, Netherlands, Taiwan, USA]",[]
4,intrusion-set--2688b13e-8e71-405a-9c40-0dee94b...,HAFNIUM,"[HAFNIUM, Operation Exchange Marauder]",2021-03-03,2022-07-06,https://attack.mitre.org/groups/G0125,7c88c982-c383-4552-90b4-cbb67ec5240f,Hafnium,[Hafnium],2021,https://apt.etda.or.th/cgi-bin/showcard.cgi?u=...,[2021-12],[2021],CN,[Information theft and espionage],None Provided,None Provided,"[T1003.001, T1560.001, T1592.004, T1583.006, T..."
